In [1]:
import pandas as pd
import random
from typing import List
import json
from copy import deepcopy
from tqdm import tqdm

In [2]:
INDUSTRY_NAME = "产业链名"
INDUSTRY_NODE = "产业链环节"
INDUSTRY_DESC = "产业链环节描述"

# df = pd.read_excel("六链五群行业分类.xlsx")
# df = df.rename(columns={"六链": INDUSTRY_NAME})

In [2]:
df = pd.read_excel("五群行业分类.xlsx", sheet_name="五群")  # 读取所有 Sheet
df.head(2)

,序号,五群,产业链环节,产业链环节描述,行业代码
0,1.0,集成电路,芯片设计,电路设计、版图设计、半导体材料、设计工具,NaN
1,NaN,集成电路,芯片制造,晶圆制造、封装测试,NaN


In [5]:
df = pd.read_excel("六链五群行业分类.xlsx", sheet_name="五群")  # 读取所有 Sheet


,序号,五群,产业链环节,产业链环节描述,行业代码
0,1.0,集成电路,芯片设计,电路设计、版图设计、半导体材料、设计工具,NaN
1,NaN,NaN,芯片制造,晶圆制造、封装测试,NaN


In [3]:
df

,序号,五群,产业链环节,产业链环节描述,行业代码
0,1.0,集成电路,芯片设计,电路设计、版图设计、半导体材料、设计工具,NaN
1,NaN,集成电路,芯片制造,晶圆制造、封装测试,NaN
2,NaN,集成电路,应用与系统集成,芯片销售、嵌入式系统开发、销售渠道,NaN
3,2.0,网络安全,安全技术研发,加密技术、防火墙技术、身份验证,NaN
4,NaN,网络安全,安全产品开发,安全软件、安全硬件,NaN
5,NaN,网络安全,安全服务提供,安全咨询、安全运维、IT服务提供商,NaN
6,4.0,电力装备,电力设备研发与设计,发电机研发、变压器研发与设计,NaN
7,NaN,电力装备,电力设备制造与组装,设备制造、部件组装、发电设备、输变电设施,NaN
8,NaN,电力装备,电力设施建设与运维,电网建设、设备运维、电力销售、维护保养,NaN
9,5.0,安全应急装备,安全应急技术研发,监测预警技术、救援技术,NaN


加载所有的行业

In [3]:
industry_cls_df = pd.read_excel("小类注释_含一二三位数代码及名称.xlsx")
industry_cls_df.head()

,小类代码2017,小类名称2017,notesum,门类代码2017,门类名称2017,大类代码2017,大类名称2017,中类代码2017,中类名称2017
0,111,稻谷种植,包括对下列稻谷的种植活动：—早籼稻：种用早籼稻、其他早籼稻；—晚籼稻：种用晚籼稻、其他晚籼稻...,A,农、林、牧、渔业,1,农业,11,谷物种植
1,112,小麦种植,包括对下列小麦的种植活动：—硬质小麦：种用硬质小麦、其他硬质小麦；—软质小麦：种用软质小麦、...,A,农、林、牧、渔业,1,农业,11,谷物种植
2,113,玉米种植,包括对下列玉米的种植活动：—白玉米：种用白玉米、其他白玉米；—黄玉米：种用黄玉米、其他黄玉米...,A,农、林、牧、渔业,1,农业,11,谷物种植
3,119,其他谷物种植,包括对下列谷物的种植活动：—谷子：硬谷子、糯谷子、其他谷子；—高粱：红粒高粱（种用红粒高粱、...,A,农、林、牧、渔业,1,农业,11,谷物种植
4,121,豆类种植,包括对下列豆类的种植活动：—大豆：黄大豆、黑大豆、青大豆、褐红大豆、双青豆、青仁乌豆、小黑豆...,A,农、林、牧、渔业,1,农业,12,豆类、油料和薯类种植


In [4]:
industry_cls_df = industry_cls_df[["小类代码2017", "小类名称2017"]]
industry_cls_df = industry_cls_df.rename(
    columns={"小类代码2017": "code", "小类名称2017": "name"}
)
industry_cls_df["code"] = industry_cls_df["code"].map(lambda x: str(x).zfill(4))
industry_cls_df.head()

,code,name
0,0111,稻谷种植
1,0112,小麦种植
2,0113,玉米种植
3,0119,其他谷物种植
4,0121,豆类种植


In [5]:
ALL_CLS_NAME = industry_cls_df["name"].values.tolist()

## build infer dataset

In [6]:
prompt_template = """
你是资深的产业专家！请先浏览下述产业环节的描述文本。
{name} {node}产业链环节：{desc}
接下来，请你逐一浏览行业分类列表：{selected_elements}，并在其中找出一眼看上去就属于上述产业链环节的行业，不需要做过渡的行业内容挖掘。
以python列表的格式输出，若没有密切相关的行业就返回空列表[]。输出样例：["xxx", "xxx"]
""".strip()

In [7]:
def select_cls(arr: List, n: int) -> List:
    """
    从 arr 中，随机筛选出 n 个元素，然后从cls中删除筛选出的元素
    """
    n = min(len(arr), n)
    if n == len(arr):
        return arr, []

    # 随机选择 n 个元素
    selected_elements = random.sample(arr, n)

    # 从 arr 中删除选中的元素
    for element in selected_elements:
        arr.remove(element)

    return selected_elements, arr

In [8]:
prompt_template.strip()

'你是资深的产业专家！请先浏览下述产业环节的描述文本。\n{name} {node}产业链环节：{desc}\n接下来，请你逐一浏览行业分类列表：{selected_elements}，并在其中找出一眼看上去就属于上述产业链环节的行业，不需要做过渡的行业内容挖掘。\n以python列表的格式输出，若没有密切相关的行业就返回空列表[]。输出样例：["xxx", "xxx"]'

In [9]:
ALL_CLS_NAME = industry_cls_df["name"].values.tolist()
prompts = []

for _, row in tqdm(df.iterrows()):
    new_CLS = deepcopy(ALL_CLS_NAME)
    while len(new_CLS) > 0:
        name, node, desc = row[INDUSTRY_NAME], row[INDUSTRY_NODE], row[INDUSTRY_DESC]
        selected_elements, new_CLS = select_cls(new_CLS, 5)
        prompt = prompt_template.format(
            selected_elements=selected_elements, name=name, node=node, desc=desc
        )
        prompts.append({"instruction": prompt, "input": "", "output": ""})

18it [00:00, 269.59it/s]


In [10]:
prompts[0]

{'instruction': '你是资深的产业专家！请先浏览下述产业环节的描述文本。\n氢能 氢气制备产业链环节：制氢原料获取、制氢技术、储氢材料\n接下来，请你逐一浏览行业分类列表：[\'陈设艺术陶瓷制造\', \'其他化工产品批发\', \'针织或钩针编织品制造\', \'农用薄膜批发\', \'地下综合管廊工程建筑\']，并在其中找出一眼看上去就属于上述产业链环节的行业，不需要做过渡的行业内容挖掘。\n以python列表的格式输出，若没有密切相关的行业就返回空列表[]。输出样例：["xxx", "xxx"]',
 'input': '',
 'output': ''}

In [11]:
def save_json(obj, file):
    with open(file, "w") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

In [12]:
save_json(prompts, "data/industry_cls.json")

In [13]:
!head -5 data/industry_cls.json

[
  {
    "instruction": "你是资深的产业专家！请先浏览下述产业环节的描述文本。\n氢能 氢气制备产业链环节：制氢原料获取、制氢技术、储氢材料\n接下来，请你逐一浏览行业分类列表：['陈设艺术陶瓷制造', '其他化工产品批发', '针织或钩针编织品制造', '农用薄膜批发', '地下综合管廊工程建筑']，并在其中找出一眼看上去就属于上述产业链环节的行业，不需要做过渡的行业内容挖掘。\n以python列表的格式输出，若没有密切相关的行业就返回空列表[]。输出样例：[\"xxx\", \"xxx\"]",
    "input": "",
    "output": ""
